In [3]:
import tensorflow as tf
import os
import numpy as np
import cv2
import csv
from tqdm import tqdm

In [4]:
def parse_data(filename, label) :
    img_read = tf.io.read_file(filename)
    img_decode = tf.image.decode_jpeg(img_read, channels=3)
#     img = tf.image.resize(img_decode, [227,227])
    return img_decode, label

def create_dataset(filenames, labels, batch_size) :
    dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
    dataset = dataset.map(parse_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(filenames))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

def read_labels(path) :
    img_classes = os.listdir(path)
    total_len = len(os.listdir(path+img_classes[0])) + len(os.listdir(path+img_classes[1])) + \
    len(os.listdir(path+img_classes[2]))+ len(os.listdir(path+img_classes[3])) + \
    len(os.listdir(path+img_classes[4]))+ len(os.listdir(path+img_classes[5]))+\
    len(os.listdir(path+img_classes[6]))

    names = []
    labels = np.zeros((total_len, len(img_classes)))
    j=0
    for i, img_class in enumerate(img_classes) :
        for img in os.listdir(os.path.join(path, img_class)) :
            names.append(os.path.join(path, img_class, img))
            labels[j][i] = 1
            j+=1
        
    return names, labels


batch_size = 16
images, labels = read_labels('./train/')
dataset = create_dataset(images, labels, batch_size)

In [11]:
effib0 =  tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(227,227,3))
effib0.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 227, 227, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 227, 227, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 229, 229, 3)  0           normalization_1[0][0]            
_____________________________________________________________________________________

In [12]:
from keras import Sequential, layers, Model
model = Sequential()
model.add(effib0)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 8, 8, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 8967      
Total params: 4,058,538
Trainable params: 4,016,515
Non-trainable params: 42,023
_________________________________________________________________


In [13]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=metrics)

In [14]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=6)

Epoch 1/6
107/107 [==============================] - 431s 4s/step - loss: 0.6659 - accuracy: 0.7839 - precision: 0.8572 - recall: 0.7038 - AUC: 0.9624
Epoch 2/6
107/107 [==============================] - 418s 4s/step - loss: 0.2826 - accuracy: 0.9122 - precision: 0.9323 - recall: 0.8916 - AUC: 0.9909
Epoch 3/6
107/107 [==============================] - 416s 4s/step - loss: 0.1935 - accuracy: 0.9452 - precision: 0.9543 - recall: 0.9340 - AUC: 0.9942
Epoch 4/6
107/107 [==============================] - 414s 4s/step - loss: 0.1600 - accuracy: 0.9488 - precision: 0.9556 - recall: 0.9370 - AUC: 0.9969
Epoch 5/6
107/107 [==============================] - 415s 4s/step - loss: 0.1232 - accuracy: 0.9582 - precision: 0.9631 - recall: 0.9529 - AUC: 0.9981
Epoch 6/6
107/107 [==============================] - 417s 4s/step - loss: 0.1550 - accuracy: 0.9470 - precision: 0.9555 - recall: 0.9352 - AUC: 0.9966


In [15]:
from tqdm import tqdm
model.save_weights('programmers_v3.ht')
test_path = './test/0'

with open('test_answer4.csv', 'w' , newline='') as f:
    wr = csv.writer(f)
    wr.writerow(['','answer value'])
    for i, image in tqdm(enumerate(os.listdir(test_path))) :
        img = cv2.imread(os.path.join(test_path, image))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, dsize=(227,227))
        img = tf.reshape(img, (-1, img.shape[0], img.shape[1], img.shape[2]))

        pred = model.predict(img)
        answer = np.argmax(pred[0])
        wr.writerow([i, answer])
    

350it [00:25, 13.98it/s]
